In [1]:
import pandas as pd
import numpy as np
import pymupdf
from dotenv import load_dotenv
import os
from collections import defaultdict

In [ ]:
excel_to_pdf_mapping ={
    #Input:Output #(Excel:PDF)
    'N': 'No',
    'Y': 'Yes',
    'Yes': 'Y',
    'No': 'N',
    'N#2FA': 'N/A',
    'Very Important': 'VI',
    'Important': 'I',
    'Considered': 'C',
    'Not Considered': 'NC',

    "private nonprofit": "Private#20#28nonprofit#29",
    "private (nonprofit)": "Private#20#28nonprofit#29",
    "proprietary": "Proprietary",

    "coeducational college": "Coeducational#20college",
    "men's college": "Men's#20college",
    "womens college": "Women's#20college",

    "semester": "SEM",
    "quarter": "QTR",
    "trimester": "TRI",
    "continuous": "CON",
    "differentiated": "DFR",
    "other": "Other",

    "High school diploma is required and GED is accepted": "GED",
    "High school diploma is required and GED is not accepted": "REQ",
    "High school diploma or equivalent is not required": "NON",

    "Required to be considered for admission":'ADMS_REQ',
    "Recommended":"ADMS_REC",
    "Required for some":"ADMS_RFS#20",
    "Not required for admission, but considered if submitted":"ADMS_CONSIDER#20",
    "Not considered for admission, even if submitted":"ADMS_NOT_USED",
}



In [ ]:
# Load environment variables
load_dotenv()
excel_file_path = os.getenv('excel_file_path') 
checkbox_mapping_file_path = os.getenv('checkbox_mapping_file_path')

# Read the Excel file
df = pd.read_excel(
    excel_file_path,
    sheet_name="Answer Sheet",
)
doc = pymupdf.open('CDS_2025-2026-PDF_Template_copy.pdf')

checkbox_button_group = defaultdict(list)
non_checkbox_button_group = defaultdict(list)
updated_fields = [] # To track which fields were updated and determined if any not updated correctly
all_fields = [] 
all_fields_options = {}


for page in doc:#doc:

    # Load all fillable pdf items, and categorize as checkbox or fill in 
    for widget in page.widgets():
        
        pdf_field_name = widget.field_name
        all_fields.append(pdf_field_name)
        
        # Here
        all_fields_options[pdf_field_name] = widget.button_states() if widget.field_type == pymupdf.PDF_WIDGET_TYPE_RADIOBUTTON else None
        #0 Collect PDF field tag corresponding value from excel
        try:
            excel_match = df.loc[df["US News PDF Tag"] == pdf_field_name, "Answer"].iloc[0]
        

            #1 Skip empty fields
            if pd.isna(excel_match):
                updated_fields.append(pdf_field_name) # Count as updated even if skipped. Trying to track which ones we have data for but did not update correctly
                continue
        
            #2 Checkbox button
            elif widget.field_type == pymupdf.PDF_WIDGET_TYPE_RADIOBUTTON:
                # if pdf_field_name in missing_fields:
                #     print('widget name: ' + widget.field_name)
                #     # print('widget state: ' + str(widget.field_value))
                #     print('widget options: ' + str(button_option))
                #     print('Excel match: ' + str(excel_match))

                excel_match = str(excel_match).strip()
                button_option = widget.button_states()['normal'] # list such as ['Public Institution', 'Off']
                if excel_match in button_option:
                    try:
                        widget.field_value = 'On'
                        widget.update()
                        updated_fields.append(pdf_field_name)
                    except Exception as e:
                        print(f"Error updating radio button field {pdf_field_name} with value {excel_match}: {e}")
                else:
                    # Try mapping Excel value to PDF value, then try updating again
                        # PDF labeling is inconsistent, sometimes yes is "Yes" and sometimes "Y"
                        # Above try block attempts direct match first, this block attempts mapped match
                    excel_match = excel_to_pdf_mapping.get(excel_match, excel_match)
                    if excel_match in button_option:
                        try:
                            widget.field_value = 'On'
                            widget.update()
                            updated_fields.append(pdf_field_name)
                        except Exception as e:
                            print(f"Error updating radio button field {pdf_field_name} with value {excel_match}: {e}")
                    # else:
                    #     print('no match for radio button:')
                
            #3 Fillable text/number fields
            else:
                excel_match = str(excel_match).strip()
                # print(f"{pdf_field_name}: {excel_match}")
                try:
                    widget.field_value = excel_match
                    widget.update()
                    updated_fields.append(pdf_field_name)
                except Exception as e:
                    print(f"Error updating field {pdf_field_name} with value {excel_match}: {e}")
        except IndexError:
            print(f"No match found for field: {pdf_field_name}")

missing_fields = [field for field in all_fields if field not in updated_fields]
print("Fields not updated (missing data or update error):")
for field in missing_fields:
    print(field)

doc.save(r"CDS_2025-2026-PDF_Template_filled3.pdf")
doc.close()

C:\Users\ywe4kw\AppData\Local\Temp\ipykernel_8804\2350829374.py:25: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_fields["PDF Field Name"].append(pdf_field_name)


TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [ ]:
checkbox_button_group

defaultdict(list,
            {'CDS_RESPONSE': [Widget:(field_type=RadioButton script=None),
              Widget:(field_type=RadioButton script=None),
              Widget:(field_type=RadioButton script=None)],
             'MAIN_INST_CONTROL': [Widget:(field_type=RadioButton script=None),
              Widget:(field_type=RadioButton script=None),
              Widget:(field_type=RadioButton script=None),
              Widget:(field_type=RadioButton script=None)],
             'MAIN_STUDENT_BODY': [Widget:(field_type=RadioButton script=None),
              Widget:(field_type=RadioButton script=None),
              Widget:(field_type=RadioButton script=None),
              Widget:(field_type=RadioButton script=None)],
             'MAIN_CALENDAR': [Widget:(field_type=RadioButton script=None),
              Widget:(field_type=RadioButton script=None),
              Widget:(field_type=RadioButton script=None),
              Widget:(field_type=RadioButton script=None),
              Widge

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [ ]:
# for page in doc:
#     for widget in page.widgets():
#         pdf_field_name = widget.field_name
        
#         matches = df.loc[df["US News PDF Tag"] == pdf_field_name, "Answer"]
#         if not matches.empty:
#             value = matches.iloc[0]
            
#             # Convert to string and handle NaN safely
#             if pd.isna(value):
#                 value = ""
#             else:
#                 value = str(value)

#                 # if value.lower() in ['yes', 'y', 'true', '1']:
#                 #     value = 'Yes'
#                 # elif value.lower() in ['no', 'n', 'false', '0']:
#                 #     value = 'No'
#                 # else:
#                 #     value = 'N/A'

#             widget.field_value = value
#             widget.update()
#         else:
#             print(f"No match found for field: {widget.field_name}")


# doc.save(r"CDS_2025-2026-PDF_Template_filled3.pdf")

yay
yay
yay
No match found for field: URL_DESTINATION
No match found for field: URL_DESTINATION


KeyError: 0

In [ ]:
# for page in doc:
#     for widget in page.widgets():
#         if widget.field_type == pymupdf.PDF_WIDGET_TYPE_RADIOBUTTON:
            
            
#             print(
#                 "Name:", widget.field_name,
#                 # "| Type:", widget.field_type,
#                 "| Value:", widget.field_value,
#                 "| States:", widget.button_states()
#             )



Name: CDS_RESPONSE | Value: Yes | States: {'normal': ['Off', 'Yes'], 'down': ['Off', 'Yes']}
Name: CDS_RESPONSE | Value: Off | States: {'normal': ['No', 'Off'], 'down': ['No', 'Off']}
Name: CDS_RESPONSE | Value: Off | States: {'normal': ['N#2FA', 'Off'], 'down': ['N#2FA', 'Off']}
Name: MAIN_INST_CONTROL | Value: Off | States: {'normal': ['Off', 'Public'], 'down': ['Off', 'Public']}
Name: MAIN_INST_CONTROL | Value: Off | States: {'normal': ['Off', 'Private#20#28nonprofit#29'], 'down': ['Off', 'Private#20#28nonprofit#29']}
Name: MAIN_INST_CONTROL | Value: Off | States: {'normal': ['Off', 'Proprietary'], 'down': ['Off', 'Proprietary']}
Name: MAIN_STUDENT_BODY | Value: Off | States: {'normal': ['Coeducational#20college', 'Off'], 'down': ['Coeducational#20college', 'Off']}
Name: MAIN_STUDENT_BODY | Value: Off | States: {'normal': ["Men's#20college", 'Off'], 'down': ["Men's#20college", 'Off']}
Name: MAIN_STUDENT_BODY | Value: Off | States: {'normal': ['Off', "Women's#20college"], 'down': ['O

In [ ]:
# checkbox_mapping.loc[checkbox_mapping['Variable']==pdf_field_name,'Yes No N/A']

2    NaN
Name: Yes No N/A, dtype: object

In [ ]:
# pdf_field_name

'MAIN_INST_CONTROL'

In [ ]:
# # Troubleshooting

# doc = pymupdf.open('CDS_2025-2026-PDF_Template_filled.pdf')

# def print_page(page):
#     for widget in page.widgets():
#         print(f"Field Name: {widget.field_name}, Field Value: {widget.field_value}")
#         matches = df.loc[df["US News PDF Tag"] == widget.field_name, "Answer"]
#         # print(matches)

# page = doc[1]
# print_page(page)
# doc.close()

Field Name: CDS_NAME, Field Value: 
Field Name: CDS_LASTNAME, Field Value: 
Field Name: CDS_TITLE, Field Value: 
Field Name: CDS_OFFICE, Field Value: Institutional Research & Analytics
Field Name: CDS_LINE1, Field Value: 
Field Name: CDS_LINE2, Field Value: 
Field Name: CDS_LINE3, Field Value: 
Field Name: CDS_CITY, Field Value: 
Field Name: CDS_STATE, Field Value:  
Field Name: CDS_ZIPCODE, Field Value: 
Field Name: CDS_COUNTRY, Field Value: 
Field Name: CDS_PHONE, Field Value: 
Field Name: CDS_EMAIL, Field Value: 
Field Name: CDS_RESPONSE, Field Value: Off
Field Name: CDS_URL, Field Value: 
Field Name: CDS_ITEMS, Field Value: 
Field Name: CDS_RESPONSE, Field Value: No
Field Name: CDS_RESPONSE, Field Value: Off


In [ ]:
# from collections import defaultdict

# doc = pymupdf.open("CDS_2025-2026-PDF_Template_filled.pdf")

# field_pages = defaultdict(list)

# for page_num, page in enumerate(doc, start=1):
#     for widget in page.widgets():
#         field_pages[widget.field_name].append(page_num)

# doc.close()

# count2 = 0
# count3 = 0
# count4 = 0
# for field_name, pages in field_pages.items():
#     if len(pages) == 2:
#         count2 += 1
#     elif len(pages) == 3:
#         count3 += 1
#     elif len(pages) == 4:
#         count4 += 1
#     # else:
#     if len(pages) > 1:
#         print(f"{field_name}")
#         # print(f"{field_name}: {len(pages)} occurrences on pages {pages}")
# print(f"Fields with 2 occurrences: {count2}"   
#       f"\nFields with 3 occurrences: {count3}"
#       f"\nFields with 4 occurrences: {count4}")

CDS_RESPONSE
URL_DESTINATION
MAIN_INST_CONTROL
MAIN_STUDENT_BODY
MAIN_CALENDAR
AD_WAIT
WAITLIST_RANK
WAITLIST_INFO_STUD
WAITLIST_INFO_SCH
AD_HS_REQ_ALL
AD_COL_PREP
AD_OPEN_MOST
AD_OPEN_T_CHECK
AD_OPEN
Q111_1
Q111_2
Q111_3
Q111_4
Q111_5
Q111_6
Q112_1
Q112_2
Q112_3
Q112_4
Q112_5
Q112_6
Q112_7
Q112_8
Q112_9
Q112_11
Q112_12
Q112_13
ADMS
EXAM_CODE_S1A
EXAM_CODE_ACT
EXAM_CODE_SAT
AP_TEST_ADVISE
AP_FEE
AP_FEE_WAIVE
AP_FEE_ONLINE
AP_FEE_ONLINE_WAIVE
AP_DL_FRSH
AP_ACCEPT_OTH
HOUS_DEPOSIT_REFUND
AD_DEFER
AD_EAD
AD_EDEC
AD_EACT
AP_EACT_RESTRICT
AD_TFER
AD_TFER_CRDT
AD_TFER_CRDT_MIN
REQ_CODE_HS_TRANSCRIPT
REQ_CODE_CG_TRANSCRIPT
REQ_CODE_ESSAY
REQ_CODE_INTERVIEW
REQ_CODE_TEST_SCORES
REQ_CODE_GDSTAND
AD_TFER_OPEN_AD
AD_TFER_CRDT_ACE
AD_TFER_CLEP_SUBJECT
AD_TFER_CRDT_DANTES
AD_TFER_CRDT_VET
ROTC_ARMY
ROTC_NAVY
ROTC_AF
TUIT_VARY_YEAR_STUD
TUIT_VARY_INST_PROG
ACAD_YR
Fields with 2 occurrences: 9
Fields with 3 occurrences: 26
Fields with 4 occurrences: 24


In [ ]:
# # C7 table for field name matching
# c7_df = pd.read_excel(
#     excel_file_path,
#     sheet_name="CDS-C",
#     usecols="B:F",
#     skiprows=99,   # skip rows 1–98
#     nrows=21       # rows 99–119 inclusive (119 - 99 + 1)
# )

# c7_df.head(20)


,Academic,Very Important,Important,Considered,Not Considered
0,Rigor of secondary school record,X,NaN,NaN,NaN
1,Class rank,X,NaN,NaN,NaN
2,Academic GPA,X,NaN,NaN,NaN
3,Standardized test scores,NaN,NaN,X,NaN
4,Application Essay,NaN,X,NaN,NaN
5,Recommendation(s),NaN,X,NaN,NaN
6,Nonacademic,Very Important,Important,Considered,Not Considered
7,Interview,NaN,NaN,NaN,X
8,Extracurricular activities,NaN,X,NaN,NaN
9,Talent/ability,NaN,X,NaN,NaN
